<a href="https://colab.research.google.com/github/saurabh1deshpande/eva7/blob/main/S4%20-%20Assignment%20QnA/assignment_4_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(1, 4, 3, padding=1) #input -1x28x28 Output - 4x28x28 
        # BN applied after this
        self.conv2 = nn.Conv2d(4, 8, 3, padding=1) #input -4x28x28  Output - 8x28x28 
        # BN applied after this
        self.pool1 = nn.MaxPool2d(2, 2) #input -8x28x28 Output - 8x14x14 
        # drop out applied after this
        self.conv3 = nn.Conv2d(8, 16, 3, padding=1) #input -8x14x14 Output - 16x14x14 
        # BN applied after this
        self.conv4 = nn.Conv2d(16, 32, 3, padding=1)#input -16x14x14  Output - 32x14x14  
        # BN applied after this
        self.pool2 = nn.MaxPool2d(2, 2) #input -32x14x14 Output - 32x7x7 
        # dropout applied after this
        self.conv5 = nn.Conv2d( 32, 64, 3) #input -32x7x7 Output - 64x5x5 
        # BN applied after this
        self.conv6 = nn.Conv2d(64, 10, 1) #input -64x5x5 Output - 10x5x5 
        self.gap = nn.AvgPool2d(5) # input - 10x5x5 Output -10x1x1
       

    def forward(self, x):
        x = F.batch_norm(F.relu(self.conv1(x)),running_mean = 0.1307 * torch.ones(4).to(device),running_var = 0.3081* torch.ones(4).to(device), training=True)
        x = F.batch_norm(F.relu(self.conv2(x)),running_mean = 0.1307 * torch.ones(8).to(device),running_var = 0.3081* torch.ones(8).to(device), training=True)
        x = self.pool1(x)
        x = F.dropout2d(x,p=0.25)

        x = F.batch_norm(F.relu(self.conv3(x)),running_mean = 0.1307 * torch.ones(16).to(device),running_var = 0.3081* torch.ones(16).to(device), training=True)
        x = F.batch_norm(F.relu(self.conv4(x)),running_mean = 0.1307 * torch.ones(32).to(device),running_var = 0.3081* torch.ones(32).to(device), training=True)
        x = self.pool2(x)
        x = F.dropout2d(x,p=0.25)

        # x = F.relu(self.conv5(x))
        x = F.batch_norm(F.relu(self.conv5(x)),running_mean = 0.1307 * torch.ones(64).to(device),running_var = 0.3081* torch.ones(64).to(device), training=True)
        x = F.relu(self.conv6(x))
        x = self.gap(x)

        x = x.view(-1, 10)
        return F.log_softmax(x)


In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 28, 28]              40
            Conv2d-2            [-1, 8, 28, 28]             296
         MaxPool2d-3            [-1, 8, 14, 14]               0
            Conv2d-4           [-1, 16, 14, 14]           1,168
            Conv2d-5           [-1, 32, 14, 14]           4,640
         MaxPool2d-6             [-1, 32, 7, 7]               0
            Conv2d-7             [-1, 64, 5, 5]          18,496
            Conv2d-8             [-1, 10, 5, 5]             650
         AvgPool2d-9             [-1, 10, 1, 1]               0
Total params: 25,290
Trainable params: 25,290
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.18
Params size (MB): 0.10
Estimated Total Size (MB): 0.28
---------------------------------------------

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    print (f'--------epoch-{epoch}-------')
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

--------epoch-1-------


loss=0.3111923336982727 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.63it/s]



Test set: Average loss: 0.2894, Accuracy: 9316/10000 (93%)

--------epoch-2-------


loss=0.12034612149000168 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 21.10it/s]



Test set: Average loss: 0.1545, Accuracy: 9609/10000 (96%)

--------epoch-3-------


...


Test set: Average loss: 0.0519, Accuracy: 9851/10000 (99%)

--------epoch-19-------


loss=0.023839695379137993 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.76it/s]



Test set: Average loss: 0.0478, Accuracy: 9864/10000 (99%)

--------epoch-20-------


loss=0.06415826082229614 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.76it/s]



Test set: Average loss: 0.0513, Accuracy: 9856/10000 (99%)

